In [6]:
import os

In [7]:
%pwd

'e:\\text-summarizer\\research'

In [8]:
%ls

 Volume in drive E is New Volume
 Volume Serial Number is 444A-66E8

 Directory of e:\text-summarizer\research

19-10-2024  08:54 PM    <DIR>          .
12-10-2024  11:01 PM    <DIR>          ..
19-10-2024  09:30 PM             4,129 01_data_ingestion.ipynb
19-10-2024  08:42 PM           184,640 Text_Summarization.ipynb
12-10-2024  11:22 PM             2,487 trials.ipynb
               3 File(s)        191,256 bytes
               2 Dir(s)  90,862,305,280 bytes free


In [9]:
os.chdir("../")

In [10]:
%pwd

'e:\\text-summarizer'

In [11]:
%ls

 Volume in drive E is New Volume
 Volume Serial Number is 444A-66E8

 Directory of e:\text-summarizer

12-10-2024  11:01 PM    <DIR>          .
12-10-2024  10:38 AM    <DIR>          .github
12-10-2024  09:56 AM             3,301 .gitignore
12-10-2024  10:40 AM                 0 app.py
12-10-2024  10:38 AM    <DIR>          config
12-10-2024  10:40 AM                 0 Dockerfile
12-10-2024  09:56 AM             1,094 LICENSE
12-10-2024  10:04 PM    <DIR>          logs
12-10-2024  11:02 PM                89 main.py
12-10-2024  10:40 AM                 0 params.yaml
19-10-2024  08:49 PM               265 README.md
12-10-2024  09:49 PM               252 requirements.txt
19-10-2024  08:54 PM    <DIR>          research
12-10-2024  09:44 PM               799 setup.py
12-10-2024  09:50 PM    <DIR>          src
12-10-2024  11:01 PM                89 tempCodeRunnerFile.py
12-10-2024  10:40 AM             1,397 template.py
              11 File(s)          7,286 bytes
               6 Dir(s)  9

In [12]:
os.chdir("src")

In [13]:
%ls

 Volume in drive E is New Volume
 Volume Serial Number is 444A-66E8

 Directory of e:\text-summarizer\src

12-10-2024  09:50 PM    <DIR>          .
12-10-2024  11:01 PM    <DIR>          ..
12-10-2024  10:04 PM    <DIR>          textSummarizer
12-10-2024  09:50 PM    <DIR>          textSummarizer.egg-info
               0 File(s)              0 bytes
               4 Dir(s)  90,862,305,280 bytes free


In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path 

In [18]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [20]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [21]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [27]:
%cd ../

e:\text-summarizer


In [28]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-20 12:52:15,551: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-20 12:52:15,559: INFO: common: yaml file: params.yaml loaded successfully]


[2024-10-20 12:52:15,561: INFO: common: created directory at: artifacts]
[2024-10-20 12:52:15,565: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-20 12:52:18,827: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3A7A:2EAC5D:235512:2DBEA2:6714AFA5
Accept-Ranges: bytes
Date: Sun, 20 Oct 2024 07:22:23 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10233-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1729408943.755943,VS0,VE696
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
